**Разбор инстумента docker-compose**

Позиционируется как инстумент, который следующая ступень абстрации над docker - инстумент который позволяет организовать взаимодейсвие контейнеров между собой.

# YAML

Язык разметки, с помощью которого задается поведение docker-compose.

YAML позволяет описывать структуры данных которые состоят из списков и словарей:
- каждый элемент списка начинается с символа `-`;
- каждый новый ключ в словаре задатся так `<ключ>:`;
- шаблоны (якоря) - возможность создать ссылку на некоторую сущность и потом использовать её в произвольном месте структуры;
- вложенность структуры формируется отступами.

Далее на примерах станет понятнее.

В python существует библиотека yaml, которая позволяет пребразовывать yaml разметку в соответсвующие структуры данных python.

In [1]:
import yaml

В следующем примере создается ключ словарь с ключами `names`, `ages` под которыми скрываются соответсвующие списки.

In [2]:
yamls_str = \
"""
names:
    - peter
    - olga
ages:
    - 22
    - 18
"""

yaml.full_load(yamls_str)

{'names': ['peter', 'olga'], 'ages': [22, 18]}

Заметим, что между `<ключ>:` и `<значение>` обязательно должен быть пробел. Вот, для сравнения, правильно созданный словарь под ключом `postgres` и ошибочно созданный под ключом `clickhouse`.

In [3]:
yamls_str = \
"""
postgres:
    user: postgres_app_user
    password: postgres_app_password
    host: postgres_host
    port: 5432
clickhouse:
    host:clickhouse_host
    user:clickhouse_app_user
    db:clickhouse_app_db
    password:clickhouse_app_password
"""

yaml.full_load(yamls_str)

{'postgres': {'user': 'postgres_app_user',
  'password': 'postgres_app_password',
  'host': 'postgres_host',
  'port': 5432},
 'clickhouse': 'host:clickhouse_host user:clickhouse_app_user db:clickhouse_app_db password:clickhouse_app_password'}

Инетестно то, что для того, что по умолчанию yaml игнорирует все переносы на новую строку. Для того, чтобы справиться с этим исполюзуется:
- `|` после имени ключа заставил yaml "видеть" перевод строки;
- `>` после имени ключа воткнет перенос строки в конец занчения.

Так в примене ниже `test1` и `test2` с точки срения программы читающей yaml не отличаются. А вот `test3` и `test4` получают в некоторых местах служебный `\n`.

In [4]:
yamls_str = \
"""
test1: peter olga
test2:
    peter
    olga
test3: |
    peter
    olga
test4: >
    peter olga
"""

yaml.full_load(yamls_str)

{'test1': 'peter olga',
 'test2': 'peter olga',
 'test3': 'peter\nolga\n',
 'test4': 'peter olga\n'}

Ну и для примера покажем как сформировать список словарей:

In [5]:
yamls_str = \
"""
- name: peter
  age: 22
- name: olga
  age: 18
"""

yaml.full_load(yamls_str)

[{'name': 'peter', 'age': 22}, {'name': 'olga', 'age': 18}]

Якоря создаются следующим образом:
- В сущности на которую ссылаются задают `&<обозначение ссылки>`;
- Когда эту сущность надо вставить куда-то используется синтаксис `<<: *<обозначение ссылки>`.

В примере далее были описаны свойства junior-a некоторой компании, а затем созданы две сушности которым были переданы свойсва этих junior-ов. 

In [6]:
yamls_str = \
"""
junior:
    &junior
    position: junior
    salary: 55000

Peter:
    <<: *junior
Olga:
    <<: *junior
"""

yaml.full_load(yamls_str)

{'junior': {'position': 'junior', 'salary': 55000},
 'Peter': {'position': 'junior', 'salary': 55000},
 'Olga': {'position': 'junior', 'salary': 55000}}

# Команды `docker-compose`

Тут будут рассмотрены самые полезные случаи для работы с коммандой `docker-compose`. Для примера используется `docker-compose.yaml` приложенный в той-же папке, что и этот notebook.

### `up` - поднять приложение

Команда `up` используется для того, чтобы поднять приложение спользующее `docker-compose`. Выполняется обязательно в той папке в которой лежит `yaml` описывающий приложение.

Опции:

- `d` - запустит в фоновом режиме - терминал останеться под управлением пользователя.

Так, следующий пример показывает, что до вызова `docker-comporse up` в docker нет не контейнеров ни вольюмов, а после, все это появляется.

In [22]:
%%bash
echo '=======запущенные контейнеры========='
docker ps
echo '===========доступные volume=========='
docker volume ls

echo '==========запускаю приложение==========='
docker-compose up -d &> /dev/null

echo '=======запущенные контейнеры========='
docker ps
echo '===========доступные volume=========='
docker volume ls

=======запущенные контейнеры=========
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
===========доступные volume==========
DRIVER    VOLUME NAME
==========запускаю приложение===========
=======запущенные контейнеры=========
CONTAINER ID   IMAGE      COMMAND                  CREATED        STATUS                  PORTS                                   NAMES
774fdb645f35   postgres   "docker-entrypoint.s…"   1 second ago   Up Less than a second   5432/tcp                                docker_compose-db-1
13c0514f5fbb   adminer    "entrypoint.sh php -…"   1 second ago   Up Less than a second   0.0.0.0:80->8080/tcp, :::80->8080/tcp   docker_compose-adminer-1
===========доступные volume==========
DRIVER    VOLUME NAME
local     docker_compose_pg_vol


### `down` - положить приложение

Опять же требуется вызывать из папки в которой лежит `yaml` описывающий приложение.

Опции:
- `v` - удалит все volume созданные при поднятии этого приложения.

In [21]:
%%bash
docker-compose up -d &> /dev/null
docker-compose down &> /dev/null

echo '============без опции -v============'
docker volume ls
docker volume rm $(docker volume ls -q) > /dev/null 2>&1


docker-compose up -d &> /dev/null
docker-compose down -v &> /dev/null
echo '============опция -v============'
docker volume ls

============без опции -v============
DRIVER    VOLUME NAME
local     docker_compose_pg_vol
============опция -v============
DRIVER    VOLUME NAME


# Создание `docker-compose` файла

Здается с помощью описанного выше языка `yaml`. Далее будем обсуждать ключи которые могут быть использованы и для чего они могут быть использованы.

### Ключ `volumes`

Задает volumes.

Каждый вложенный ключ создаст volume.

```
volumes:
    <volume1>:
        name: <volume name>
    <volume2>:
    ...
```

У каждого volume можно задать поле `name` (а можно и не задавать) которое укажет имя volume при поднятии контейнера. Интерестно то, что если не задать имя volume то docker-compose сам его сгенерирует, отталкиваясь от названия папки в которой запущен.